This file contains the code to find the most suitable K value for the KNN filling algorithm
Since Logistic regression performs and worst among all algorithm, we don't use it here

In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
# Cross Validation Classification Accuracy
from pandas import read_csv
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn import preprocessing 
from sklearn.impute import KNNImputer
# from sklearn.neighbors import KNeighborsClassifier
#import faiss
from numpy import isnan
#import matplotlib.pyplot as plot
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score
from sklearn import tree
import statistics
import time
import warnings
warnings.filterwarnings('ignore')
# plotting errorbar graph for visual representation of the performance of the classifiers
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import random
from xgboost import XGBClassifier
import shutil
import os
from os import path
# import ipynb.fs.full.our_functions_library as flib
import our_functions_library as flib
from our_evaluate_sepsis_score import evaluate_performance
import numba
from numba import jit, njit, cuda

In [ ]:
# Global initialization, get all the data from file and generate all the needed variable
filename = '../../raw_data/raw_data_400.csv' # use raw dataset
#filename = 'data.csv' # use raw dataset
originalData = read_csv(filename) # read csv data into DataFrame var raw
print("Data size:",originalData.shape)
Original_Uniq_ID= np.unique(originalData['Patient_id']) 
Uniq_ID = Original_Uniq_ID.copy()
print('Patient id size:',len(Uniq_ID))
X_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_columns = ['Patient_id', 'SepsisLabel']

# To test if these 3 columns is useful
# originalData = originalData.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)
# X_columns.remove('Unit1')
# X_columns.remove( 'Unit2')
# X_columns.remove( 'EtCO2')

# Initialize the empty array X_train, X_test, y_train, y_test
X_train = pd.DataFrame(columns = X_columns)
X_test = pd.DataFrame(columns = X_columns)
y_train = pd.DataFrame(columns = y_columns)
y_test = pd.DataFrame(columns = y_columns)

# store the X & y train/test in array for easy access
globaldata = []
globaldata.append(X_train)
globaldata.append(y_train)
globaldata.append(X_test)
globaldata.append(y_test)
# Below are the lists for KNN results

KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []

KNN_F1Score_mean = []
KNN_F1Score_std = []

KNN_auroc_mean = []
KNN_auprc_mean = []

KNN_accuracy_mean = []
KNN_accuracy_std = []

KNN_positiveprediction_mean = []
KNN_sepsisLabelRatio_mean = []

KNN_baseline_mean = [ ]
KNN_total_Time= [ ]


fillmethod =""  

In [ ]:
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 8
random.seed(seed)
if(np.array_equal(Original_Uniq_ID,Uniq_ID) ):
    print("They are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs    
else:
    print("They are not the same. Uniq_id has already been shuffled.")    
print("The original uniq id set is:\n",Original_Uniq_ID[1:20])
print("Uniq id:************************************")
print("The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49 345...\n",Uniq_ID[1:20])

In [ ]:
# #fill the missing data with one function
# print(filename)
# #print(originalData)
# if (originalData.isnull().values.any()):
#     print('There is data missing in the original data set')
# dataByPatient = originalData.groupby('Patient_id') # data grouped by Patient_id

In [ ]:
def KNN_reset():
    #KNN with different K size, 
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_sepsisLabelRatio_mean.clear() #array for storing the mean of the sepsislabel ratio (predict/true)
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()

def generateTrainDataSet(test_patienIds, X_train, y_train):
    train_data= originalData[~(originalData.Patient_id.isin(test_patienIds))]
    X_train = train_data[X_columns]
    y_train = train_data[y_columns]
    return X_train, y_train
 
def generateTestDataSet(patienIds, X_test, y_test):
    #print("test_patienIds: \n", patienIds)
    test_data = originalData[originalData['Patient_id'].isin(patienIds)]
    X_test = test_data[X_columns]
    y_test = test_data[y_columns]
    return X_test, y_test

# def CalculateSOFAScore(X_train_impute, X_test_impute):
#     pass

In [ ]:
# Code for K-fold algorithm, Before this step the missing data has been filled, 
def KFold_patient(model, KforKFold=10,KforKNN=5, fillmethod=""):
    start = time.time()# time indicator for how long the Kfold func takes
    X_train = globaldata[0]
    y_train = globaldata[1]
    X_test = globaldata[2]
    y_test = globaldata[3]
    print("Datasize",len(Uniq_ID))
    print("K Fold of ",KforKFold ," folds with KNN =",KforKNN)
    
    # initialisation of the array for storing the different intermediate results
    accuracy_model = []
    F1Score_model = []
    baseline_model= []
    auroc_model = []
    auprc_model = []
    physio_accuracy_model = []
    f_measure_model = []
    utility_score_model = []
    mean_train = 0
    positivepredictions = []
    predictionsRatio = []

    #The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
    idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups
    #print("Patients number: ", len(Uniq_ID)) #print('idSets arrays',idSets)
    #This for loop is for Kfold, calculating the results for K times
    for i in range(KforKFold):
        start1 = time.time()
        X_train, X_test, y_train, y_test = flib.clearAllDatasets(X_train, X_test, y_train, y_test) #first clear all the datasets
        print("for the",i+1,"th iteration",idSets[i])   
        t1 = time.time()
        X_test,y_test = generateTestDataSet(idSets[i], X_test, y_test)
        print("Time for splitting id to test:", round(time.time()- t1,2))
        
        t2= time.time()
        X_train, y_train = generateTrainDataSet(idSets[i], X_train, y_train)
        print("Time for splitting id to train:", round(time.time()- t2,2))

        #printDataset()
        #Now the train and test dataset is generated
        #we can begin to train the model wit the training set and evaaulate the performance with the test sett   
#         X_train = X_train.drop('time', 1) X_test = X_test.drop('time', 1)
        X_train=X_train.astype('float64')
        X_test=X_test.astype('float64')    
        YTest_copy = y_test  # variable of joining the filled data (X) and Y (Train_output) 
        patientID_ytest = y_test['Patient_id']
        y_train = y_train.drop('Patient_id', 1)
        y_test = y_test.drop('Patient_id', 1)      
        y_train=y_train.astype('float64')
        y_test=y_test.astype('float64')
        #print('YTest',YTest_copy.head())
        
        #fill the missing data
        if X_train.isnull().values.any() or X_test.isnull().values.any() :
            print("X_train or X_test contains NaN values, KNN/mean is performed.")
            #if there is missing value
            X_train_impute, X_test_impute, fillmethod = flib.KNNfilling(X_train, X_test, KforKNN, fillmethod)
            #X_train_impute, X_test_impute,fillmethod = flib.MeanFilling(X_train,X_test, fillmethod)
            #check the missing data   
            if np.isnan(X_train_impute).any() or np.isnan(X_test_impute).any() :
                print("X_train_impute or X_test_impute still contains NaN values")        
            else:
                print("X_train_impute or X_test_impute have all been filled ")  
        else:
            print("X_train or X_test have all been filled ")
            X_train_impute = X_train
            X_test_impute = X_test
                 
        #Scale the data， uncomment this part for logistic regression only, in other case, put them in comment
        model_name = type(model).__name__
        if (model_name == 'LogisticRegression'):
            print("scaled!")
            scaler = preprocessing.StandardScaler()
            scaler.fit(X_train_impute)
            X_train_impute = scaler.transform(X_train_impute)
            X_test_impute = scaler.transform(X_test_impute)

        #fit the model and predict
        model.fit(X_train_impute, y_train)
        y_predicted = model.predict(X_test_impute)   
        y_predicted_probobility = model.predict_proba(X_test_impute) 
        #transfer the outpu and evalute it
        y_labels = y_test.astype(int).to_numpy()
        y_predicted = y_predicted.astype(int)
        if(len(y_predicted_probobility[0]) ==2 ):
            y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
        else:
            print("columns of y_predicted_probobility is: ",len(y_predicted_probobility[0]))
            y_predicted_probobility =  y_predicted_probobility.round(4)
        #print(y_labels)      
        auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance(y_labels, y_predicted, y_predicted_probobility,patientID_ytest)
        auroc_model.append(round(auroc,4))
        auprc_model.append(round(auprc,4))
        physio_accuracy_model.append(round(physio_accuracy,4))
        f_measure_model.append(round(f_measure,4))
        utility_score_model.append(round(utility_score,4))
        positivepredictions.append(np.sum(y_predicted))
        predictionsRatio.append(np.sum(y_predicted)/np.sum(y_test))
        horizontal_stack = pd.concat([y_test, y_predicted], axis=1)


        #take down the results
        accuracy_model.append((accuracy_score(y_test, y_predicted, normalize=True)*100).round(2))
        F1Score_model.append((f1_score(y_test, y_predicted)*100).round(2))
        baseline_model.append(round( (1 - float(y_test.mean()) )*100 , 2 ))
        print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
        #baseline_model.append(((1-y_test.mean())*100).round(2))
        print("The number of 1 (SepsisLabel) in this prediction: ", np.sum(y_predicted))
        end1 = time.time()
        print("Time spent in this KFold iteration",round(end1-start1,2),"sec.\n")
        print("******************************************************************")
    print('Accuracy model:', accuracy_model)
    print('F1_score model:', F1Score_model)
    print('Baseline model:', baseline_model)
    
    print("\nEvaluation parameters of the utiltiy evaluation function:")
    print('auroc of model:', auroc_model)
    print('auprc of model:', auprc_model)
    print('Utility accuracy of model:', accuracy_model)
    print('utility F1 of model:', f_measure_model)
    print('Utility score of model:', utility_score_model)
    
    KNN_UtilityScore_mean.append(np.mean(utility_score_model))
    KNN_UtilityScore_std.append(np.std(utility_score_model))
    
    KNN_F1Score_mean.append(np.mean(f_measure_model))
    KNN_F1Score_std.append(np.std(f_measure_model))
    
    KNN_auroc_mean.append(np.mean(auroc_model))
    KNN_auprc_mean.append(np.mean(auprc_model))
    
    KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
    KNN_accuracy_std.append(np.std(physio_accuracy_model))
    
    KNN_positiveprediction_mean.append(np.mean(positivepredictions))
    KNN_sepsisLabelRatio_mean.append(np.mean(predictionsRatio))

    KNN_baseline_mean.append(np.mean(baseline_model))
    totalTime=round(time.time()- start,2)
    KNN_total_Time.append(totalTime)
    
    print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")


In [ ]:
def Print_DATA():
    print("KNN_UtilityScore_mean",KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std",KNN_UtilityScore_std)
    print("KNN_F1Score_mean",KNN_F1Score_mean)
    print("KNN_F1Score_std",KNN_F1Score_std)
    print("KNN_auroc_mean",KNN_auroc_mean)
    print("KNN_auprc_mean",KNN_auprc_mean)
    print("KNN_accuracy_mean",KNN_accuracy_mean)
    print("KNN_accuracy_std",KNN_accuracy_std)
    print("KNN_positiveprediction_mean",KNN_positiveprediction_mean)
    print("KNN_sepsisLabelRatio_mean", KNN_sepsisLabelRatio_mean)
    print("KNN_baseline_mean",KNN_baseline_mean)
    print("KNN_total_time",KNN_total_Time)

In [17]:
#Here is how to train the model
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2)
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2)
#AdaBoostModel = AdaBoostClassifier(random_state=2)

In [18]:
#fill the missing data with one function
print(filename)
#print(originalData)
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func')
#data,fillmethod = flib.linearFillingAll(originalData,1, True)#forwardfilling + fill other missing data with 0
#dataByPatient = data.groupby('Patient_id') # data grouped by Patient_id

../../raw_data/raw_data_400.csv
Data isn´t all filled before K-Fold Func


In [19]:
originalData.columns

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel', 'Patient_id', 'time'],
      dtype='object')

In [20]:
missing = originalData.isna().sum()*100/len(originalData)
print(missing)

cols=list(missing[missing<90].index)
print(cols)
print(np.shape(cols))
col = cols.copy()
print(col)
X_columns = col
data=originalData[cols]
print(data.head())

dataByPatient = data.groupby('Patient_id') # data grouped by Patient_id

HR                   9.727652
O2Sat               12.816002
Temp                65.930414
SBP                 14.255929
MAP                 11.825645
DBP                 30.453479
Resp                15.461298
EtCO2               95.810529
BaseExcess          94.344540
HCO3                95.934324
FiO2                92.194423
pH                  93.067501
PaCO2               94.377118
SaO2                96.631483
AST                 98.175658
BUN                 93.158718
Alkalinephos        98.201720
Calcium             94.513943
Chloride            95.706281
Creatinine          93.888455
Bilirubin_direct    99.765442
Glucose             82.870732
Lactate             97.419859
Magnesium           93.738598
Phosphate           96.097211
Potassium           90.819651
Bilirubin_total     98.377639
TroponinI           98.996612
Hct                 91.132395
Hgb                 92.611415
PTT                 97.061506
WBC                 93.575710
Fibrinogen          99.400573
Platelets 

In [21]:
# print("Logistic Regression")
KFold_patient(decisionTreeModel,10, 5)

Datasize 400
K Fold of  10  folds with KNN = 5
for the 1 th iteration [ 74 171 275 215 388 212 112 226  61 252 232 312 105  66 143 199 270  27
 343 237 380 324  57 247 274 266 373  89 283  85 156 188  39  43   4 272
 213  91 299 323]
Time for splitting id to test: 0.0
Time for splitting id to train: 0.0
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 


TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid

In [ ]:
Print_DATA()

In [ ]:
# print("Logistic Regression")
KFold_patient(XGBoostModel,10, 5)

In [38]:
# print("Logistic Regression")
KFold_patient(XGBoostModel,10, 5)

K Fold of  10  folds with KNN = 5
for the 1 th iteration [ 74 171 275 215 388 212 112 226  61 252 232 312 105  66 143 199 270  27
 343 237 380 324  57 247 274 266 373  89 283  85 156 188  39  43   4 272
 213  91 299 323]
Time for splitting id to test: 0.66
Time for splitting id to train: 22.18
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 

y_test size: (1405, 1) 1´s in y_test SepsisLabel    49.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  49
Time spent in this KFold iteration 72.06 sec.

******************************************************************
for the 2 th iteration [285  63 302 288 164 256  67 160 240 138 179 307 150 242 391   8 169 118
 356 394 222  19 332 261 392 124  10 139 235 140   5  41 244 385  17 330
 358 314 165 289]
Time for splitting id to test: 0.31
Time for splitting id to train: 4.81
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_i

In [17]:
Print_DATA()

KNN_UtilityScore_mean [0.8763799999999999, 0.8763799999999999]
KNN_UtilityScore_std [0.04571071646780435, 0.04571071646780435]
KNN_F1Score_mean [1.0, 1.0]
KNN_F1Score_std [0.0, 0.0]
KNN_auroc_mean [0.0, 1.0]
KNN_auprc_mean [0.0, 1.0]
KNN_accuracy_mean [1.0, 1.0]
KNN_accuracy_std [0.0, 0.0]
KNN_positiveprediction_mean [22.7, 22.7]
KNN_baseline_mean [97.124, 97.124]
KNN_total_time [113.18, 127.0]


In [ ]:
## drop features with 90% missing (decisionTreee)
# Accuracy model: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
# F1_score model: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
# Baseline model: [98.47, 97.58, 98.21, 95.94, 98.42, 98.48, 95.81, 95.2, 98.54, 94.59]

# Evaluation parameters of the utiltiy evaluation function:
# auroc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# auprc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Utility accuracy of model: [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]
# utility F1 of model: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
# Utility score of model: [1.0, 0.9126, 0.8344, 0.8503, 0.8654, 0.8571, 0.8515, 0.8783, 0.8571, 0.8571]

# Total Time spent in  KFold function 113.18 sec.
# KNN_UtilityScore_mean [0.8763799999999999]
# KNN_UtilityScore_std [0.04571071646780435]
# KNN_F1Score_mean [1.0]
# KNN_F1Score_std [0.0]
# KNN_auroc_mean [0.0]
# KNN_auprc_mean [0.0]
# KNN_accuracy_mean [1.0]
# KNN_accuracy_std [0.0]
# KNN_positiveprediction_mean [22.7]
# KNN_baseline_mean [97.124]
# KNN_total_time [113.18]

In [54]:
Print_DATA()

KNN_UtilityScore_mean [0.10571]
KNN_UtilityScore_std [0.13807516395065406]
KNN_F1Score_mean [0.10476]
KNN_F1Score_std [0.10757065770924709]
KNN_auroc_mean [0.0]
KNN_auprc_mean [0.0]
KNN_accuracy_mean [0.9001999999999999]
KNN_accuracy_std [0.07986740261207949]
KNN_positiveprediction_mean [75.8]
KNN_baseline_mean [97.124]
KNN_total_time [134.2]


In [35]:
## 200

# Accuracy model: [94.39, 89.65, 77.34, 83.25, 93.85, 93.94, 88.73, 92.87, 97.38, 91.34]
# F1_score model: [0.0, 2.53, 5.58, 2.94, 0.0, 0.0, 18.75, 24.66, 0.0, 11.11]
# Baseline model: [98.47, 97.58, 98.21, 95.94, 98.42, 98.48, 95.81, 95.2, 98.54, 94.59]

# Evaluation parameters of the utiltiy evaluation function:
# auroc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# auprc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Utility accuracy of model: [94.39, 89.65, 77.34, 83.25, 93.85, 93.94, 88.73, 92.87, 97.38, 91.34]
# utility F1 of model: [0.0, 0.0253, 0.0558, 0.0294, 0.0, 0.0, 0.1875, 0.2466, 0.0, 0.1111]
# Utility score of model: [-0.0923, -0.0583, 0.0301, -0.0208, -0.0837, -0.0857, 0.2554, 0.2509, -0.0229, 0.0562]

# Total Time spent in  KFold function 305.26 sec.


# Accuracy model: [94.39, 89.65, 77.34, 83.25, 93.85, 93.94, 88.73, 92.87, 97.38, 91.34]
# F1_score model: [0.0, 2.53, 5.58, 2.94, 0.0, 0.0, 18.75, 24.66, 0.0, 11.11]
# Baseline model: [98.47, 97.58, 98.21, 95.94, 98.42, 98.48, 95.81, 95.2, 98.54, 94.59]

# Evaluation parameters of the utiltiy evaluation function:
# auroc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# auprc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Utility accuracy of model: [94.39, 89.65, 77.34, 83.25, 93.85, 93.94, 88.73, 92.87, 97.38, 91.34]
# utility F1 of model: [0.0, 0.0253, 0.0558, 0.0294, 0.0, 0.0, 0.1875, 0.2466, 0.0, 0.1111]
# Utility score of model: [-0.0923, -0.0583, 0.0301, -0.0208, -0.0837, -0.0857, 0.2554, 0.2509, -0.0229, 0.0562]

# Total Time spent in  KFold function 109.83 sec.

In [14]:
# print("Decision tree")
#KFold_patient(randomForestModel,10, 2)

In [15]:
# findBestKforKNN(logisticRegressionModel,10,1,10)
# findBestKforKNN(XGBoostModel,10,110,120)

In [16]:
# findBestKforKNN(decisionTreeModel,10,1,10)
# os.system('\a')

In [17]:
displayCurrentResult(1,9)

NameError: name 'displayCurrentResult' is not defined